<a href="https://colab.research.google.com/github/ArbabKhan-sudo/Natural-Language-Processing/blob/main/Using_GPT_model_for_language_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Transfomers are the reason why you see AI everywhere!

In [1]:
!pip install transformers datasets evaluate sacrebleu
!pip install  sentencepiece sacremoses -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 54.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ...

In [2]:
from datasets import load_dataset
books = load_dataset("opus_books", "de-en")

Generating train split:   0%|          | 0/51467 [00:00<?, ? examples/s]

In [3]:
books = books["train"].train_test_split(test_size=0.2)

In [4]:
books['train'][0]

{'id': '48623',
 'translation': {'de': ',,Na, du hast drum gebeten, und ich will\'s dir geben; aber wenn\'s dir nicht gefällt, darfst du niemand Vorwürfe machen als dir selbst."',
  'en': '"Now you\'ve asked for it, and I\'ll give it to you, because there ain\'t anything mean about me; but if you find you don\'t like it, you mustn\'t blame anybody but your own self."'}}

In [5]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [6]:
#from transformers import T5ForConditionalGeneration, T5Tokenizer, DataCollatorForSeq2Seq, AdamWeightDecay, TFAutoModelForSeq2SeqLM

#checkpoint = "google/t5-v1_1-small"
#tokenizer = T5Tokenizer.from_pretrained(checkpoint)

In [7]:
source_lang = "de" # Deutsch (German)
target_lang = "en" # English
prefix = "translate German to English: "

In [8]:
def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [9]:
#
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/41173 [00:00<?, ? examples/s]

Map:   0%|          | 0/10294 [00:00<?, ? examples/s]

In [10]:
tokenized_books

DatasetDict({
    train: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 41173
    })
    test: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10294
    })
})

In [11]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

In [12]:
from transformers import AdamWeightDecay
from transformers import TFAutoModelForSeq2SeqLM

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [13]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")

In [14]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_books["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_books["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [15]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

In [16]:
# (Specially Crafted Adversarial Examples for Robust Evaluation of Bilingual Evaluation Understudy) metric.

import evaluate
metric = evaluate.load("sacrebleu")
# BLEU measures the quality of a generated translation by calculating the precision. comparing it to one or more reference translations.

In [17]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [18]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_test_set)

In [19]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3)

Epoch 1/3
2573/2573 [==============================] - 797s 301ms/step - loss: 3.2698 - val_loss: 2.8285
Epoch 2/3
2573/2573 [==============================] - 773s 301ms/step - loss: 3.0257 - val_loss: 2.7141
Epoch 3/3
2573/2573 [==============================] - 772s 300ms/step - loss: 2.9188 - val_loss: 2.6455


In [20]:
text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."

In [27]:
from transformers import pipeline

tokenzzz= tokenizer(text, return_tensors="pt", padding=True, truncation=True)
translator = pipeline("translation", model=model, tokenizer=tokenzzz)

translator.tokenizer

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/__init__.py:986: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


{'input_ids': tensor([[13959,  1566,    12,  2379,    10,   312,  1744,  2687,   698,  1438,
            28, 23383,    18, 12304,    53,  8710,     5,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}